In [2]:
import os
import pandas as pd
import plotly.express as px
import progressbar
import json
import numpy as np
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from functions import open_product_zip_file, files_to_df_extended, delete_empty_rows
from functions import remove_punctuations, extract_most_similar
from nltk.tokenize import word_tokenize
pd.set_option('display.max_colwidth', None)

## Get the paths and build table names for iteration

In [3]:
data_path = '../src/data'
mapping_corpus_path = data_path + r'/product/lspc2020_to_tablecorpus'
mapping_corpus_path_2 = data_path + r'/product/lspc2020_to_tablecorpus/Cleaned'
table_corpus_path = data_path + r'/product/product_top100/cleaned'
table_corpus_path_with_id = data_path + r'/product/product_top100/cleaned/with_id'
table_corpus_path2 = data_path + r'/product/product_minimum3/cleaned/with_id'

In [4]:
zip_files_mapping = [file for file in os.listdir(mapping_corpus_path_2) if file.endswith('.json.gz')]
zip_files_tables = [file for file in os.listdir(table_corpus_path) if file.endswith('.json.gz')]

## Match the number dictionaries with the information about the brand

In [4]:
df_electronics_cleaned = pd.read_json(mapping_corpus_path_2 + '/cleaned_electronics_all_brands', compression='gzip', orient='records', lines=True)
df_clothes_cleaned = pd.read_json(mapping_corpus_path_2 + '/cleaned_clothes_all_brands', compression='gzip', orient='records', lines=True)

In [5]:
df_large = pd.read_json(os.path.join(mapping_corpus_path_2, 'df_large_matched'), compression='gzip', orient='records', lines=True)

In [9]:
df_joined_electronics = df_large.merge(df_electronics_cleaned, left_on=['table_id','row_id'], right_on = ['table_id','row_id'], how='left')

In [10]:
df_joined_clothes = df_large.merge(df_clothes_cleaned, left_on=['table_id','row_id'], right_on = ['table_id','row_id'], how='left')

In [11]:
df_joined_electronics.to_json(mapping_corpus_path_2 + '/joined_electronics_v2', compression='gzip', orient='records', lines=True)

In [12]:
df_joined_clothes.to_json(mapping_corpus_path_2 + '/joined_clothes_v2', compression='gzip', orient='records', lines=True)

In [29]:
df_joined_electronics = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_electronics_v2'), compression='gzip', orient='records', lines=True)

In [7]:
df_joined_clothes = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_clothes_v2'), compression='gzip', orient='records', lines=True)

In [8]:
df_joined_electronics

,cluster_id,url,row_id,table_id,Valid,name,description,brand_x,brand_y
0,59178314,https://www.cultureindoor.com/796-irrigation-arrosage-tuyau-pe-20mm-semi-rigide-rouleau-100m.html,1561,Product_cultureindoor.com_September2020.json.gz,1,"irrigation ,arrosage Tuyau PE 20mm semi rigide rouleau 100m","Tuyau PE 20mm semi rigide rouleau 100m, idéal pour les irrigations de jardins et arrosage automatique, retrouvez toute l'irrigation chez culture indoor le leader du jardinage et de l'hydroponie en france et en europe",FRANCE ARROSAGE,None
1,70692685,https://www.culturekings.com/products/new-era-new-era-new-orleans-pelicans-nba-940-a-frame-snapback-black,3893,Product_culturekings.com_September2020.json.gz,1,New Era New Orleans Pelicans NBA 9FORTY A-Frame Snapback Black,New Era is back at it again this season with another unique instalment of the 940 silhouette. This time around Culture Kings has the Worldwide Exclusive on these so you can be assured that you won't find them anywhere else. Culture Kings have worked closely with New Era to design a hat that features a slightly higher crown for increased comfort and added style points. The highest quality cotton twill fabric has been used for extreme comfort and durability and the shape is so unique that you just won't find this on any other hat. - Curved Brim - Extra room at the crown- Embroidered team logo- Rear snapback closure,None,None
2,395606,https://www.culturekings.com/products/new-era-x-nrl-west-tigers-aframe-charcoal-orange,3894,Product_culturekings.com_September2020.json.gz,1,New Era X NRL West Tigers Aframe Charcoal/orange,"The Iconic New Era is the premium siloute of headwear worldwide. The official Major league baseball on field hat, as well as now NRL. It's fit is can be worn with flat visor or curved, The best quality hat in the world.",None,None
3,3361797,https://www.customizedgirl.com/design/2605608/The+Real+Baseball+Moms,9318,Product_customizedgirl.com_September2020.json.gz,1,The Real Baseball Moms (2XL),"Are you a real baseball mom fan? Customize a cute design spoof on the \""real housewives\"" show. Add your team's name and your athlete's name.",CustomizedGirl.com,None
4,235755,https://www.customizedgirl.com/design/2613423/Mentally+Dating+Ronaldo,9319,Product_customizedgirl.com_September2020.json.gz,1,Mentally Dating Ronaldo (2XL),"Mentally dating Christiano Ronaldo. Yeah girl, me too. He literally has changed the game of soccer into something women watch just to see him. That hair, those abs, the smile. It's too much to handle. If I was the goalie, I'd let him score every single time.",CustomizedGirl.com,None
...,...,...,...,...,...,...,...,...,...
24688354,8653717,https://shop-list.com/women/utuwa-tatara/sg3232-03,1620,Product_shop-list.com_September2020.json.gz,1,vZXEC h[ yAv[gZbg(Vf) v[g Mtg ,yPrincessRoyalDreams-vZXCh[-zyAv[gZbg(Vf)(v[g2_̃ZbgłB)fBYj[EvZX݂hCEFfBOhC[WBFȂXg[[ƋɁAK𑡂܂B܂AϔȂ̂ŁAMtgƂĂ߂łBFB₲Ƒւ̃v[gMtgɂł傤B+++V[Y֘Ai+++EyAOXZbg(Vf)EyAXeOXZbg(Vf)EXeOX&amp;v[gZbg(AG)EXeOX&amp;v[gZbg(vcF)TCY21~H2.3cmeʁ[ގ({)@v[g͉_gpĂ܂BӂƓdqWOKHNGNG̑t̎戵mFB݌ɂ̏i͓XʓX܂ł̔Ă܂B݌ɐ̍XV͐sĂ܂A󋵂ɂĂ͏ii؂ƂȂ邱Ƃ܂B܂i񂹂ɂ肨͂܂łɂԂꍇ܂Bqlɂ͕KA܂Aꃁ[J[ŕi؂ꓙ̗Rɂח肪ȂAׂ͓xꍇ̓LZĂꍇ܂Ƃ炩߂B,None,None
24688355,22258637,https://shop-list.com/women/youmotto/famille-ltm,1621,Product_shop-list.com_September2020.json.gz,1,t@~[ OgC}bg80~60cm[gC }bg gC^[ Jg[ _ i` IV AC{[uE n O CeA 􂦂ېOK tH ~ `RȊO],yiڍׁzyގzpC/|GXe100yTCYz80~60cmyYzyɂĂ̂ӁzEŏ̐͗VітAڐ肷邱Ƃ܂̂ŁAK̕ƕĐĂB͎኱kނƂ܂Agpɂ͂܂BEh@iȂȂ߃h܂ށj͐΂ɂgpɂȂȂłBpCiёj̔`̘č݂ƂȂ܂̂Ŏ􂢂ߒv܂ByӁzE܂肽ŕۊǂĂׁAiɐ܂ȂtĂ邱Ƃ܂Bx󂵂Ă܂ƁA܂ȂڗȂȂ܂BEg̃j^[̐ݒɂAF̌قȂ邱Ƃ܂BB,None,None
24688356,41434658,https://shop.agwaycapecod.com/products/019014710959/adult-small-breed-dog-food-5-lb,126,Product_agwaycapecod.com_September2020.json.gz,1,"Adult Small Breed Dog Food, 5-Lb.","Eukanuba, 5 LB, Adult Small Breed Dry Dog Food, Has A Higher Amount Of Fat To Meet The Unique Energy Levels, Kibble Size Suits Their Metabolisms & Stomach Size, Promotes Active, Daily Living With Antioxidants For Immune Health, Fiber For Healthy Digestion, Fatty Acids For Coat &

In [9]:
df_joined_clothes

,cluster_id,url,row_id,table_id,Valid,name,description,brand_x,brand_y
0,59178314,https://www.cultureindoor.com/796-irrigation-arrosage-tuyau-pe-20mm-semi-rigide-rouleau-100m.html,1561,Product_cultureindoor.com_September2020.json.gz,1,"irrigation ,arrosage Tuyau PE 20mm semi rigide rouleau 100m","Tuyau PE 20mm semi rigide rouleau 100m, idéal pour les irrigations de jardins et arrosage automatique, retrouvez toute l'irrigation chez culture indoor le leader du jardinage et de l'hydroponie en france et en europe",FRANCE ARROSAGE,None
1,70692685,https://www.culturekings.com/products/new-era-new-era-new-orleans-pelicans-nba-940-a-frame-snapback-black,3893,Product_culturekings.com_September2020.json.gz,1,New Era New Orleans Pelicans NBA 9FORTY A-Frame Snapback Black,New Era is back at it again this season with another unique instalment of the 940 silhouette. This time around Culture Kings has the Worldwide Exclusive on these so you can be assured that you won't find them anywhere else. Culture Kings have worked closely with New Era to design a hat that features a slightly higher crown for increased comfort and added style points. The highest quality cotton twill fabric has been used for extreme comfort and durability and the shape is so unique that you just won't find this on any other hat. - Curved Brim - Extra room at the crown- Embroidered team logo- Rear snapback closure,None,None
2,395606,https://www.culturekings.com/products/new-era-x-nrl-west-tigers-aframe-charcoal-orange,3894,Product_culturekings.com_September2020.json.gz,1,New Era X NRL West Tigers Aframe Charcoal/orange,"The Iconic New Era is the premium siloute of headwear worldwide. The official Major league baseball on field hat, as well as now NRL. It's fit is can be worn with flat visor or curved, The best quality hat in the world.",None,None
3,3361797,https://www.customizedgirl.com/design/2605608/The+Real+Baseball+Moms,9318,Product_customizedgirl.com_September2020.json.gz,1,The Real Baseball Moms (2XL),"Are you a real baseball mom fan? Customize a cute design spoof on the \""real housewives\"" show. Add your team's name and your athlete's name.",CustomizedGirl.com,None
4,235755,https://www.customizedgirl.com/design/2613423/Mentally+Dating+Ronaldo,9319,Product_customizedgirl.com_September2020.json.gz,1,Mentally Dating Ronaldo (2XL),"Mentally dating Christiano Ronaldo. Yeah girl, me too. He literally has changed the game of soccer into something women watch just to see him. That hair, those abs, the smile. It's too much to handle. If I was the goalie, I'd let him score every single time.",CustomizedGirl.com,None
...,...,...,...,...,...,...,...,...,...
24688354,8653717,https://shop-list.com/women/utuwa-tatara/sg3232-03,1620,Product_shop-list.com_September2020.json.gz,1,vZXEC h[ yAv[gZbg(Vf) v[g Mtg ,yPrincessRoyalDreams-vZXCh[-zyAv[gZbg(Vf)(v[g2_̃ZbgłB)fBYj[EvZX݂hCEFfBOhC[WBFȂXg[[ƋɁAK𑡂܂B܂AϔȂ̂ŁAMtgƂĂ߂łBFB₲Ƒւ̃v[gMtgɂł傤B+++V[Y֘Ai+++EyAOXZbg(Vf)EyAXeOXZbg(Vf)EXeOX&amp;v[gZbg(AG)EXeOX&amp;v[gZbg(vcF)TCY21~H2.3cmeʁ[ގ({)@v[g͉_gpĂ܂BӂƓdqWOKHNGNG̑t̎戵mFB݌ɂ̏i͓XʓX܂ł̔Ă܂B݌ɐ̍XV͐sĂ܂A󋵂ɂĂ͏ii؂ƂȂ邱Ƃ܂B܂i񂹂ɂ肨͂܂łɂԂꍇ܂Bqlɂ͕KA܂Aꃁ[J[ŕi؂ꓙ̗Rɂח肪ȂAׂ͓xꍇ̓LZĂꍇ܂Ƃ炩߂B,None,None
24688355,22258637,https://shop-list.com/women/youmotto/famille-ltm,1621,Product_shop-list.com_September2020.json.gz,1,t@~[ OgC}bg80~60cm[gC }bg gC^[ Jg[ _ i` IV AC{[uE n O CeA 􂦂ېOK tH ~ `RȊO],yiڍׁzyގzpC/|GXe100yTCYz80~60cmyYzyɂĂ̂ӁzEŏ̐͗VітAڐ肷邱Ƃ܂̂ŁAK̕ƕĐĂB͎኱kނƂ܂Agpɂ͂܂BEh@iȂȂ߃h܂ށj͐΂ɂgpɂȂȂłBpCiёj̔`̘č݂ƂȂ܂̂Ŏ􂢂ߒv܂ByӁzE܂肽ŕۊǂĂׁAiɐ܂ȂtĂ邱Ƃ܂Bx󂵂Ă܂ƁA܂ȂڗȂȂ܂BEg̃j^[̐ݒɂAF̌قȂ邱Ƃ܂BB,None,None
24688356,41434658,https://shop.agwaycapecod.com/products/019014710959/adult-small-breed-dog-food-5-lb,126,Product_agwaycapecod.com_September2020.json.gz,1,"Adult Small Breed Dog Food, 5-Lb.","Eukanuba, 5 LB, Adult Small Breed Dry Dog Food, Has A Higher Amount Of Fat To Meet The Unique Energy Levels, Kibble Size Suits Their Metabolisms & Stomach Size, Promotes Active, Daily Living With Antioxidants For Immune Health, Fiber For Healthy Digestion, Fatty Acids For Coat &

In [15]:
# df_tables_count=df_join.groupby('cluster_id').count()
# df_tables_count[df_tables_count['brand']>2]

## Get information about electronic clusters and train model

In [30]:
df_grouped_electronics = df_joined_electronics.groupby('cluster_id').count()
# only look at clusters that have at least one brand associated
df_set_electronics = df_grouped_electronics[df_grouped_electronics['brand_y']>0].reset_index()[['cluster_id','table_id']].rename(columns={'table_id':'Amount'})

In [31]:
# We discard all clusters with less than 2 entries, cause we cannot match anything there, so 1,6 million clusters remain
df_set_electronics=df_set_electronics[df_set_electronics['Amount']>1]
df_15_electronics=df_set_electronics[df_set_electronics['Amount']>15]
df_15_electronics

,cluster_id,Amount
7,1040,21
26,6076,19
27,6443,25
29,6505,16
30,6690,20
...,...,...
108804,76869317,60
110646,78260404,80
112351,79512161,16
112994,80097153,60


In [32]:
#merge brand name to cluster amount
df_cluster_brand = df_15_electronics[df_15_electronics['Amount']<200].merge(df_joined_electronics.dropna()[['cluster_id','brand_y']].drop_duplicates('cluster_id', keep='last'), left_on=['cluster_id'], right_on = ['cluster_id'], how='left')
df_cluster_brand

,cluster_id,Amount,brand_y
0,1040,21,nikon
1,6076,19,canon
2,6443,25,canon
3,6505,16,apple
4,6690,20,cisco
...,...,...,...
405,76869317,60,NaN
406,78260404,80,NaN
407,79512161,16,NaN
408,80097153,60,samsung


In [69]:
#clean product column and lowercase
df_joined_electronics=df_joined_electronics.dropna(subset = ['name'])
df_joined_electronics['name'] = df_joined_electronics['name'].apply(lambda row: row.lower())
df_joined_electronics
#get only cluster ids with at least one brand electronics
df_compare_electronics = df_joined_electronics[df_joined_electronics['cluster_id'].isin(df_set_electronics['cluster_id'].tolist())]
#merge with set to get amount of tables per cluster in overview
df_compare_electronics = df_compare_electronics.merge(df_set_electronics, left_on=['cluster_id'], right_on = ['cluster_id'], how='left')

In [70]:
#use tokenizer for product names to get tokes for training the model
df_compare_electronics['product_tokes'] = df_compare_electronics['name'].apply(lambda row: word_tokenize(row))
#get tagged words
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(df_compare_electronics['product_tokes'])]
# build model and vocabulary
model = Doc2Vec(vector_size=50, min_count = 5, epochs = 25, dm = 0)
model.build_vocab(tagged_data)
# Train model
model.train(tagged_data, total_examples=model.corpus_count, epochs=25)

In [1]:
#look into specific clusters
#df_joined_electronics[df_joined_electronics['cluster_id']==562955][['cluster_id','name','brand_x','brand_y']]

In [74]:
#get cluster ids for basline products and with that indices of top products to use model
top_clusters_list = [1524820,47566,6076,14418,28307,33570,39040,51314,99153,215254,685416, 984421 , 1808651,2887810,34506065,47841827,620473,56116,94055, 150211,182246, 516888, 562955    ]
index_top_clusters_list=[]
for id in top_clusters_list:
    index_top_clusters_list.append(df_compare_electronics[df_compare_electronics['cluster_id']==id].index[0])

In [75]:
# get most similar products for each of the base clusters and save them if they have more than 5 tables
electronics_clusters_search=[]
for i in index_top_clusters_list:
    similar_doc = model.docvecs.most_similar(f'{i}', topn = 15)
    electronics_clusters_search.append(int(i))
    for index, similarity in similar_doc:
        if df_compare_electronics.iloc[int(index)]['Amount']>5:
            electronics_clusters_search.append(int(index))
df_electroncis_final = df_compare_electronics.iloc[electronics_clusters_search]

<ipython-input-75-5e7499ac0349>:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar(f'{i}', topn = 15)


In [76]:
df_electroncis_final.drop_duplicates('cluster_id', keep='first').to_excel("Final_Electronics_v2.xlsx")

## Cluster statistics for product category clothes

In [63]:
df_grouped_clothes = df_joined_clothes.groupby('cluster_id').count()

In [64]:
# only look at clusters that have at least one brand associated
df_set_clothes = df_grouped_clothes[df_grouped_clothes['brand_y']>0].reset_index()[['cluster_id','table_id']].rename(columns={'table_id':'Amount'})

In [65]:
# We discard all clusters with less than 2 entries, cause we cannot match anything there, so 1,6 million clusters remain
df_set_clothes=df_set_clothes[df_set_clothes['Amount']>1]
df_set_clothes

,cluster_id,Amount
0,532,4
1,560,2
2,1507,3
3,1747,4
5,2033,8
...,...,...
175641,80465508,2
175642,80467771,2
175653,80472255,4
175654,80472257,4


In [77]:
df_10_clothes=df_set_clothes[df_set_clothes['Amount']>15]
df_10_clothes

,cluster_id,Amount
657,135583,18
668,138313,17
725,148199,24
740,150932,25
785,159911,18
...,...,...
110633,48051979,17
130738,58083280,24
131754,58592784,19
170854,77602818,16


In [78]:
#merge brand name to cluster amount
df_cluster_brand_clothes = df_10_clothes[df_10_clothes['Amount']<400].merge(df_joined_clothes.dropna()[['cluster_id','brand_y']].drop_duplicates('cluster_id', keep='last'), left_on=['cluster_id'], right_on = ['cluster_id'], how='left')
df_cluster_brand_clothes

,cluster_id,Amount,brand_y
0,135583,18,tissot
1,138313,17,christian louboutin
2,148199,24,ray ban
3,150932,25,louis vuitton
4,159911,18,chopard
...,...,...,...
82,48051979,17,calvin klein
83,58083280,24,nike
84,58592784,19,nike
85,77602818,16,valentino


In [82]:
df_joined_clothes=df_joined_clothes.dropna(subset = ['name'])
#clean product column and lowercase
df_joined_clothes['name'] = df_joined_clothes['name'].apply(lambda row: row.lower())
df_joined_clothes
#get only cluster ids with at least one brand electronics
df_compare_clothes = df_joined_clothes[df_joined_clothes['cluster_id'].isin(df_set_clothes['cluster_id'].tolist())]
#merge with set to get amount of tables per cluster in overview
df_compare_clothes = df_compare_clothes.merge(df_set_clothes, left_on=['cluster_id'], right_on = ['cluster_id'], how='left')

<ipython-input-82-1ab36e755ca8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_joined_clothes['name'] = df_joined_clothes['name'].apply(lambda row: row.lower())


In [83]:
#use tokenizer for product names to get tokes for training the model
df_compare_clothes['product_tokes'] = df_compare_clothes['name'].apply(lambda row: word_tokenize(row))
#get tagged words
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(df_compare_clothes['product_tokes'])]
# build model and vocabulary
model = Doc2Vec(vector_size=50, min_count = 5, epochs = 25, dm = 0)
model.build_vocab(tagged_data)
# Train model
model.train(tagged_data, total_examples=model.corpus_count, epochs=25)

In [103]:
# have a look into the clusters for getting base clusters
#df_joined_clothes[df_joined_clothes['cluster_id']==44159446][['cluster_id','name','brand_x','brand_y']]

,cluster_id,name,brand_x,brand_y
601347,44159446,runway gold tone multi-function watch,Michael Kors,michael kors
3378332,44159446,runway gold tone multi-function watch,Michael Kors,michael kors
4080567,44159446,runway gold tone multi-function watch,Michael Kors,michael kors
4735499,44159446,runway gold tone multi-function watch,Michael Kors,michael kors
5459053,44159446,runway gold tone multi-function watch,Michael Kors,michael kors
5469494,44159446,mk6588 runway women's watch,Michael Kors,michael kors
5490903,44159446,runway gold tone multi-function watch,Michael Kors,michael kors
6252661,44159446,runway gold tone multi-function watch,Michael Kors,michael kors
7224845,44159446,runway ɫ๦ֱ,Michael Kors,None
9900404,44159446,runway gold tone multi-function watch,Michael Kors,michael kors


In [104]:
#get cluster ids and with that indices of top products to use model
top_clusters_list = [5310, 58043,104343,142594,174327, 186753,421372,677207,834201, 881202,  895708,939889, 1249086,1290229, 1852022,2459966, 2732926 , 22374915, 22374918, 26097914,44159446, 58592784, 78110534,135583,148199, 200956, 950691, 1592417,2464591]
index_top_clusters_list=[]
for id in top_clusters_list:
    index_top_clusters_list.append(df_compare_clothes[df_compare_clothes['cluster_id']==id].index[0])

In [105]:
# get most similar products for each of the base clusters and save them if they have more than 5 tables
clothes_clusters_search=[]
for i in index_top_clusters_list:
    similar_doc = model.docvecs.most_similar(f'{i}', topn = 15)
    clothes_clusters_search.append(int(i))
    for index, similarity in similar_doc:
        if df_compare_clothes.iloc[int(index)]['Amount']>5:
            clothes_clusters_search.append(int(index))
df_clothes_final = df_compare_clothes.iloc[clothes_clusters_search]

<ipython-input-105-b837bfa1a69a>:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar(f'{i}', topn = 15)


In [106]:
df_clothes_final.drop_duplicates('cluster_id', keep='first').to_excel("Final_Clothes_v2.xlsx")